In [1]:
import os
import sys

ROOT_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
EXPERIMENT_PATH = f"{ROOT_PATH}/experiments/exploration-strategy"
sys.path.insert(0, ROOT_PATH)

In [2]:
from json import load, dumps, dump
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from src.datasets.oracle import Oracle, OracleRequest, TIMEOUT
from src.datasets.data_config import HINTSETS, DOPS, HINTS, DEFAULT_HINTSET
from src.utils import get_full_plan

In [3]:
cached_oracles = {
    "JOB": Oracle(f"{ROOT_PATH}/data/processed/JOB"),
    "tpch_10gb": Oracle(f"{ROOT_PATH}/data/processed/tpch_10gb"),
    "sample_queries": Oracle(f"{ROOT_PATH}/data/processed/sample_queries"),
}

bench_to_title = {
    "JOB": "JOB",
    "sample_queries": "SQ",
    "tpch_10gb": "TPCH"
}

DEFAULT_DOP = 64

# Search Strategy

Given the vastness of the search space, even with boolean hints ($2^{\# \text{hintsets}}$), and our desire to manage the degree of parallelism (`DOP` value), we are compelled to find ways to reduce the search space. 

Initially, a **greedy algorithm** was considered, which sequentially disables hintsets. However, it was observed in practice that disabling a single operation is sometimes insufficient to achieve the desired result (if the planner considers $op_1 < op_2 < op_3$, we cannot make it use $op_3$ by simply disabling only one operation). This led to the decision to implement actions such as "disable all joins except one," making the search algorithm resemble a local search approach.

Since it is not predetermined which actions are most promising, we **parameterize** the general algorithm scheme and **empirically determine the best combinations of actions**. It is evident that the more actions we add, the more extensively we will explore, thereby potentially finding better solutions, but at the cost of increased search expenses.

The solution state (`SearchingState`) during the search is described by only two parameters - the set of disabled operations (`hintset`) and the chosen degree of parallelism (`dop`). 

The neighborhood of the solution is defined by search parameters (`SearchingSettings`). I will write more about what exactly each of them does somewhere else ...

## Search Parameterization

In [4]:
OFF_INL_HINT = 64 | 8 | 2
N_SCANS = 4
N_JOINS = 3
NL_POS = 2
assert N_SCANS + N_JOINS == len(HINTS)

### `SearchingState` and `SearchingSettings`

In [5]:
from collections import namedtuple, defaultdict

In [6]:
SearchingState = namedtuple(
    "SearchingState", 
    ["dop", "hintset"], 
    defaults=[DEFAULT_DOP, DEFAULT_HINTSET]
)

In [7]:
SearchingSettings = namedtuple(
    "SearchingSettings", 
    [
        "disable_ops",
        "decrease_dop", 
        "disable_inl",
        "boost_threshold", 
        "max_iter", 
        "avoid_duplicates",
        "use_timeout",
        "use_joined_search",
        "prioritize_neighbors",
        "force_join",
        "force_only_nl",
    ],
    defaults=[
        False, 
        False, 
        False, 
        1.0, 
        0,
        False,
        False,
        False,
        False,
        False,
        False,
    ]    
)

In [8]:
settings_pool = []
for disable_ops in [False, True]:
    for decrease_dop in [False, True]:
        for disable_inl in [False, True]:
            for max_iter in [1, 2, float("inf")]:
                for boost_threshold in [1.0, 1.1, 1.2, 1.5, 2.0]:   
                    for use_timeout in [False, True]:
                        for avoid_duplicates in [False, True]:
                            for use_joined_search in [False, True]:
                                for prioritize_neighbors in [False, True]:
                                    for force_join in [False, True]:
                                        for force_only_nl in [False, True]:
                                            settings = SearchingSettings(
                                                disable_ops=disable_ops,
                                                decrease_dop=decrease_dop, 
                                                disable_inl=disable_inl, 
                                                boost_threshold=boost_threshold,
                                                max_iter=max_iter,
                                                use_timeout=use_timeout,
                                                avoid_duplicates=avoid_duplicates,
                                                use_joined_search=use_joined_search,
                                                prioritize_neighbors=prioritize_neighbors,
                                                force_join=force_join,
                                                force_only_nl=force_only_nl,
                                            )
                                            settings_pool.append(settings)

In [9]:
print(f"THE NUMBER OF ALL SEARCHING SETTINGS IS {len(settings_pool)}")

THE NUMBER OF ALL SEARCHING SETTINGS IS 7680


### `QueryExplorer`

It is some wrapper over the `oracle` for a specific query and specific settings `SearchingSettings` parameters. Its purpose is to perform local search according to the selected settings while managing the total execution time, number of planning, and so on

In [10]:
class QueryExplorer:
    def __init__(self, query_name, oracle, settings=SearchingSettings()):
        self.oracle = oracle
        self.query_name = query_name
        self.settings = settings

        self.learning_time = 0
        self.n_executions = 0
        self.n_plannings = 0

        self.default_plan = None
        self.explored_plans = set()
        self.explored_states = set()

        self._warmup()

    def _warmup(self):
        def_request = OracleRequest(query_name=self.query_name, dop=DEFAULT_DOP, hintset=DEFAULT_HINTSET)
        def_state = SearchingState()

        ex_time = self.oracle.get_execution_time(request=def_request)
        plan_time = self.oracle.get_planning_time(request=def_request)  
        plan = self.get_plan(state=def_state, bypass=True)

        self.default_time = ex_time + plan_time
        self.learning_time += ex_time + plan_time
        self.default_plan = plan 
        self.explored_plans.add(plan)
        self.explored_states.add(def_state)
        self.n_executions += 1

    def explore_state(self, state, timeout=None, bypass=False):
        if not self.settings.use_timeout or timeout is None:
            timeout = 2 * self.default_time

        request = OracleRequest(query_name=self.query_name, dop=state.dop, hintset=state.hintset)
        ex_time = self.oracle.get_execution_time(request=request)
        plan_time = self.oracle.get_planning_time(request=request)
        e2e_time = min(timeout, ex_time + plan_time)

        if not bypass:
            self.learning_time += e2e_time
            self.explored_plans.add(self.get_plan(state=state, bypass=True))
            self.explored_states.add(state)
            self.n_executions += 1
            
        return e2e_time

    def get_plan(self, state, bypass=True):
        plan = get_full_plan(
            query_name=self.query_name, 
            oracle=self.oracle,
            dop=state.dop,
            hintset=state.hintset
        )
        request = OracleRequest(query_name=self.query_name, dop=state.dop, hintset=state.hintset)
        if not bypass:
            self.learning_time += self.oracle.get_planning_time(request=request)
            self.n_plannings += 1

        return plan

    def get_ex_time(self, dop, hintset):
        request = OracleRequest(query_name=self.query_name, dop=dop, hintset=hintset)
        return self.oracle.get_execution_time(request=request)

    def run(self):
        prev_state = None
        
        plan_to_ex_time = defaultdict(float)
        plan_to_ex_time[self.default_plan] = self.get_ex_time(dop=DEFAULT_DOP, hintset=DEFAULT_HINTSET)

        record_state = SearchingState()
        record_time = self.default_time

        it = 0
        while it < self.settings.max_iter:
            timeout = record_time/self.settings.boost_threshold

            prev_state = record_state
            neighbors = filter(lambda st: st not in self.explored_states, self.get_neighbors(state=record_state))
            
            if self.settings.prioritize_neighbors:
                neighbors = sorted(neighbors, key=lambda st: self.explore_state(state=st, bypass=True))
            
            for ngb_state in neighbors:
                if self.settings.avoid_duplicates:
                    ngb_plan = self.get_plan(state=ngb_state)
                    if ngb_plan in plan_to_ex_time: 
                        ngb_time = self.explore_state(state=ngb_state, timeout=timeout, bypass=True)
                    else:
                        ngb_time = self.explore_state(state=ngb_state, timeout=timeout)
                        plan_to_ex_time[ngb_plan] = self.get_ex_time(dop=ngb_state.dop, hintset=ngb_state.hintset) 
                else:
                    ngb_time = self.explore_state(state=ngb_state, timeout=timeout)
                
                if record_time / ngb_time > self.settings.boost_threshold:
                    record_state, record_time = ngb_state, ngb_time
                    if self.settings.prioritize_neighbors:
                        break
            
            it += 1
            if prev_state == record_state:
                break  
            
        return record_state

    def get_neighbors(self, state):
        current_dop, current_hintset = state.dop, state.hintset
        neighbors = set()

        if self.settings.use_joined_search:
            to_try_dops = DOPS
        else:
            to_try_dops = [current_dop]
        
        for dop in to_try_dops:
            if self.settings.disable_ops:
                for op_num in range(len(HINTS)):
                    neighbors.add(SearchingState(dop=dop, hintset=current_hintset | (1 << op_num)))

            if self.settings.disable_inl:
                neighbors.add(SearchingState(dop=dop, hintset=current_hintset | (OFF_INL_HINT)))

            if self.settings.decrease_dop:
                for new_dop in [new_dop for new_dop in DOPS if new_dop < dop]:
                    neighbors.add(SearchingState(dop=new_dop, hintset=current_hintset))                    
            
            if self.settings.force_join:
                for join_num in range(N_JOINS):
                    saved_scans = ((1 << N_SCANS) - 1) & current_hintset
                    only_one_join = (((1 << N_JOINS) - 1) - (1 << join_num)) << N_SCANS
                    neighbors.add(SearchingState(dop=dop, hintset=only_one_join|saved_scans))

            if self.settings.force_only_nl:
                join_num = NL_POS
                saved_scans = ((1 << N_SCANS) - 1) & current_hintset
                only_one_join = (((1 << N_JOINS) - 1) - (1 << join_num)) << N_SCANS
                neighbors.add(SearchingState(dop=dop, hintset=only_one_join|saved_scans))                    

        return neighbors        

### Step 1 - collecting info for each settings

In [11]:
e2e_times = defaultdict(dict)
learning_times = defaultdict(dict)

for settings in tqdm(settings_pool):
    
    for bench_name, oracle in cached_oracles.items():
        e2e_time, learning_time = 0, 0

        for query_name in oracle.get_query_names():
            explorer = QueryExplorer(query_name=query_name, oracle=oracle, settings=settings)
            dop, hintset = explorer.run()
            learning_time += explorer.learning_time
            request = OracleRequest(query_name=query_name, hintset=hintset, dop=dop)
            e2e_time += oracle.get_execution_time(request) + oracle.get_planning_time(request)
        
        e2e_times[bench_name][settings] = round(e2e_time/1000, 3)
        learning_times[bench_name][settings] = round(learning_time/1000, 3)

100%|██████████| 7680/7680 [06:26<00:00, 19.85it/s]


### Step 2 - collecting info for baseline and ideal case

In [12]:
best_e2e_times = {}
def_times = {}

all_states = [SearchingState(dop, hintset) for dop in DOPS for hintset in HINTSETS]

for bench_name, oracle in cached_oracles.items():
    best_e2e_time, def_time = 0, 0
    for query_name in oracle.get_query_names():
        explorer = QueryExplorer(query_name=query_name, oracle=oracle, settings=settings)
        best_state = min(all_states, key=lambda st: explorer.explore_state(st))
        best_dop, best_hintset = best_state.dop, best_state.hintset

        best_request = OracleRequest(query_name=query_name, hintset=best_hintset, dop=best_dop)
        best_e2e_time += oracle.get_execution_time(best_request) + oracle.get_planning_time(best_request)

        def_request = OracleRequest(query_name=query_name, hintset=DEFAULT_HINTSET, dop=DEFAULT_DOP)
        def_time += oracle.get_execution_time(def_request) + oracle.get_planning_time(def_request)

    best_e2e_times[bench_name] = round(best_e2e_time / 1000, 3)
    def_times[bench_name] = round(def_time / 1000, 3)

### *Which moves are the most important to get high boost*?

We will analyze results for `JOB` benchmark and just provide info for the rest ones.

In [23]:
def evaluate_settings(descr, bench_name, settings):
    e2e_time = e2e_times[bench_name][settings]
    boost = def_times[bench_name] - e2e_times[bench_name][settings]
    max_boost = def_times[bench_name] - best_e2e_times[bench_name]
    print(f"| {descr} | {100 * boost/max_boost:0.1f} % | {learning_times[bench_name][settings]:0.2f} s |")

In [14]:
bench_name = "tpch_10gb"
MAX_ITER = float("inf")

In [15]:
settings_and_descr = [
    (
        SearchingSettings(
            disable_ops=True, 
            max_iter=MAX_ITER
        ), 
        "`DISABLE_OP`"
    ),
    
    (
        SearchingSettings(
            disable_inl=True, 
            max_iter=MAX_ITER
        ), 
        "`DISABLE_INL`"
    ),
    
    (
        SearchingSettings(
            force_join=True, 
            max_iter=MAX_ITER
        ), 
        "`FORCE_JOIN`"
    ),

    (
        SearchingSettings(
            decrease_dop=True, 
            max_iter=MAX_ITER
        ), 
        "`DECREASE_DOP`"
    ),
    
    (
        SearchingSettings(
            disable_ops=True, 
            decrease_dop=True, 
            max_iter=MAX_ITER
        ), 
        "`DISABLE_OP` OR `DECREASE_DOP`"
    ),

    (
        SearchingSettings(
            disable_inl=True, 
            decrease_dop=True, 
            max_iter=MAX_ITER
        ), 
        "`DISABLE_INL` OR `DECREASE_DOP`"
    ),

    (
        SearchingSettings(
            force_join=True, 
            decrease_dop=True, 
            max_iter=MAX_ITER
        ),
        "`FORCE_JOIN` OR `DECREASE_DOP`"
    ),

    (
        SearchingSettings(
            force_join=True, 
            disable_inl=True, 
            decrease_dop=True,
            max_iter=MAX_ITER
        ),
        "(`FORCE_JOIN` OR `DISABLE_INL`  OR `DECREASE_DOP`)"
    ),

    (
        SearchingSettings(
            force_join=True, 
            disable_inl=True, 
            disable_ops=True, 
            decrease_dop=True,
            max_iter=MAX_ITER
        ),
        "(`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`)"
    ),

    (
        SearchingSettings(
            disable_ops=True, 
            use_joined_search=True, 
            max_iter=MAX_ITER
        ),
        "`DISABLE_OP` AND `CHANGE_DOP`"
    ),

    (
        SearchingSettings(
            disable_inl=True, 
            use_joined_search=True, 
            max_iter=MAX_ITER
        ),
        "`DISABLE_INL` AND `CHANGE_DOP`",
    ),

    (
        SearchingSettings(
            force_join=True, 
            use_joined_search=True, 
            max_iter=MAX_ITER
        ),
        "`FORCE_JOIN` AND `CHANGE_DOP`"
    ),

    (
        SearchingSettings(
            force_join=True, 
            disable_inl=True, 
            use_joined_search=True, 
            max_iter=MAX_ITER
        ),
        "(`FORCE_JOIN` OR `DISABLE_INL`) AND `CHANGE_DOP`"
    ),

    (
        SearchingSettings(
            force_join=True, 
            disable_inl=True, 
            disable_ops=True, 
            decrease_dop=True,
            use_joined_search=True, 
            max_iter=MAX_ITER
        ),
        "(`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) AND `CHANGE_DOP`"
    ),
]

for settings, descr in settings_and_descr:
    evaluate_settings(descr, bench_name, settings)

| `DISABLE_OP` | 91.2 % | 7611.98 s |
| `DISABLE_INL` | 29.8 % | 1175.42 s |
| `FORCE_JOIN` | 65.5 % | 2051.35 s |
| `DECREASE_DOP` | 22.0 % | 1527.04 s |
| `DISABLE_OP` OR `DECREASE_DOP` | 99.3 % | 11913.05 s |
| `DISABLE_INL` OR `DECREASE_DOP` | 50.1 % | 2814.50 s |
| `FORCE_JOIN` OR `DECREASE_DOP` | 73.4 % | 4408.37 s |
| (`FORCE_JOIN` OR `DISABLE_INL`  OR `DECREASE_DOP`) | 76.5 % | 6059.70 s |
| (`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) | 99.7 % | 16546.93 s |
| `DISABLE_OP` AND `CHANGE_DOP` | 100.0 % | 24994.40 s |
| `DISABLE_INL` AND `CHANGE_DOP` | 46.3 % | 2545.40 s |
| `FORCE_JOIN` AND `CHANGE_DOP` | 75.2 % | 5118.48 s |
| (`FORCE_JOIN` OR `DISABLE_INL`) AND `CHANGE_DOP` | 78.4 % | 9478.47 s |
| (`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) AND `CHANGE_DOP` | 100.0 % | 34287.37 s |


We observe that limiting operations on `JOB` alone can achieve only `69%` of the maximum boost. Moreover, even utilizing a local search procedure to explore all neighborhoods requires about `10,000` seconds (~`x20` the execution time of the benchmark). By both restricting operations and adjusting the degree of parallelism, we can reach around `78%`. Further gains necessitate the use of specific join-related hints. Employing both `OFF_INL` and `FORCE_JOIN` can elevate this to `92%`. However, approaching the optimum necessitates simultaneously altering joins along with `DOP`s. Unfortunately, this proves to be an extremely costly operation — a complete investigation in this mode takes about `38,000` seconds, considering the use of T/O for long queries.

At the same time, simply trying one action such as a) reducing the degree of parallelism, b) allowing only one join, or c) turning off `INL` can achieve ~ `40%`. Considering that this **only requires exploration 1-3 neighbors**, it's an exceptionally good result. The main observation, however, is that only by altering joins (`NO_INL`, `FORCE_JOIN`) together with adjusting the degree of parallelism, can one reach `94%`! And this in just `12,000` seconds compared to `38,000` for a full exploration!

### `JOB`

| `SearchingSetting` | Boost (% of optimal) | Learning Time (s) |
|-|-|-|
| `DISABLE_OP` | 69.2 % | 10149.92 s |
| `DISABLE_INL` | 38.0 % | 1115.87 s |
| `FORCE_JOIN` | 52.2 % | 1911.83 s |
| `DECREASE_DOP` | 40.8 % | 1514.32 s |
| `DISABLE_OP` OR `DECREASE_DOP` | 78.1 % | 13114.70 s |
| `DISABLE_INL` OR `DECREASE_DOP` | 74.6 % | 2951.83 s |
| `FORCE_JOIN` OR `DECREASE_DOP` | 66.2 % | 4340.27 s |
| (`FORCE_JOIN` OR `DISABLE_INL`  OR `DECREASE_DOP`) | **87.9 %** | **7374.47 s** |
| (`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) | 91.5 % | 17072.23 s |
| `DISABLE_OP` AND `CHANGE_DOP` | 82.2 % | 30877.65 s |
| `DISABLE_INL` AND `CHANGE_DOP` | 71.6 % | 2423.74 s |
| `FORCE_JOIN` AND `CHANGE_DOP` | 66.3 % | 4780.83 s |
| (`FORCE_JOIN` OR `DISABLE_INL`) AND `CHANGE_DOP` | **94.2 %** | **12112.15 s** |
| (`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) AND `CHANGE_DOP` | 98.5 % | 37748.68 s |

### `sample_queries`

| `SearchingSetting` | Boost (% of optimal) | Learning Time (s) |
|-|-|-|
| `DISABLE_OP` | 85.4 % | 12914.35 s |
| `DISABLE_INL` | 40.5 % | 1603.38 s |
| `FORCE_JOIN` | 74.5 % | 2751.21 s |
| `DECREASE_DOP` | 49.4 % | 2151.14 s |
| `DISABLE_OP` OR `DECREASE_DOP` | 88.2 % | 16663.09 s |
| `DISABLE_INL` OR `DECREASE_DOP` | 75.4 % | 3951.95 s |
| `FORCE_JOIN` OR `DECREASE_DOP` | 79.3 % | 6052.97 s |
| (`FORCE_JOIN` OR `DISABLE_INL`  OR `DECREASE_DOP`) | 85.3 % | 9715.47 s |
| (`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) | 93.5 % | 20437.88 s |
| `DISABLE_OP` AND `CHANGE_DOP` | 93.0 % | 33940.96 s |
| `DISABLE_INL` AND `CHANGE_DOP` | 59.9 % | 3084.41 s |
| `FORCE_JOIN` AND `CHANGE_DOP` | 82.6 % | 6626.23 s |
| (`FORCE_JOIN` OR `DISABLE_INL`) AND `CHANGE_DOP` | **88.0 %** | **13185.60 s** |
| (`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) AND `CHANGE_DOP` | 95.0 % | 44233.18 s |


### `TPCH`

| `SearchingSetting` | Boost (% of optimal) | Learning Time (s) |
|-|-|-|
| `DISABLE_OP` | **91.2 %** | **7611.98 s** |
| `DISABLE_INL` | 29.8 % | 1175.42 s |
| `FORCE_JOIN` | **65.5 %** | **2051.35 s** |
| `DECREASE_DOP` | 22.0 % | 1527.04 s |
| `DISABLE_OP` OR `DECREASE_DOP` | **99.3 %** | **11913.05 s** |
| `DISABLE_INL` OR `DECREASE_DOP` | 50.1 % | 2814.50 s |
| `FORCE_JOIN` OR `DECREASE_DOP` | 73.4 % | 4408.37 s |
| (`FORCE_JOIN` OR `DISABLE_INL`  OR `DECREASE_DOP`) | 76.5 % | 6059.70 s |
| (`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) | 99.7 % | 16546.93 s |
| `DISABLE_OP` AND `CHANGE_DOP` | 100.0 % | 24994.40 s |
| `DISABLE_INL` AND `CHANGE_DOP` | 46.3 % | 2545.40 s |
| `FORCE_JOIN` AND `CHANGE_DOP` | 75.2 % | 5118.48 s |
| (`FORCE_JOIN` OR `DISABLE_INL`) AND `CHANGE_DOP` | 78.4 % | 9478.47 s |
| (`FORCE_JOIN` OR `DISABLE_INL` OR `DISABLE_OP` OR `DECREASE_DOP`) AND `CHANGE_DOP` | 100.0 % | 34287.37 s |

## *Do we really need to explore all of neighbors?*

Given the semantics of `SearchingState`, it is clear that the outcomes of exploring a state depend solely on the plan they lead to. Thus, executing the same plan twice is unnecessary.

However, in practice, we have observed an interesting phenomenon: although two different states may lead to the same plan (which obviously will execute for the same amount of time), the planning time can vary significantly! Sometimes it may be beneficial to use hints (or change the level of parallelism) simply to reduce the planning time.

Let's verify this in practice:

In [16]:
def collect_info_about_plans(bench_name):
    oracle = cached_oracles[bench_name]
    for query_name in oracle.get_query_names():
        for hintset in HINTSETS:
            for dop in DOPS:
                request = OracleRequest(query_name=query_name, hintset=hintset, dop=dop)
                planning_time = oracle.get_planning_time(request=request)
                ex_time = oracle.get_execution_time(request=request)
                if ex_time >= TIMEOUT:
                    continue
                plan = QueryExplorer(query_name, oracle).get_plan(SearchingState(hintset=hintset, dop=dop))
                plan_to_times[plan].append((planning_time, query_name, ex_time, dop, hintset))

In [17]:
for bench_name in ["JOB", "sample_queries"]:
    plan_to_times = defaultdict(list)
    collect_info_about_plans(bench_name)

    plan_times_ordered_by_deviation = sorted(
        plan_to_times.values(), 
        key=lambda el : max([p_t for p_t, *info in el]) - min([p_t for p_t, *info in el])
    )

    for plan_times in plan_times_ordered_by_deviation[-3:]:
        plan_times = sorted(plan_times)
        query_name = plan_times[0][1]
        plan_time_info = f"{plan_times[0][0]/1000:0.3f}s - {plan_times[-1][0]/1000:0.3f}s"
        ex_time_info = f"{plan_times[0][2]/1000:.1f}s - {plan_times[-1][2]/1000:.1f}s"
        print(f"| `{query_name}` | {plan_time_info} | {ex_time_info} |")

| `29c` | 2.072s - 8.832s | 0.0s - 0.0s |
| `28b` | 0.595s - 8.222s | 2.6s - 2.6s |
| `29a` | 0.881s - 12.540s | 0.0s - 0.0s |
| `q17_7a164` | 1.114s - 4.547s | 52.6s - 52.6s |
| `q17_7a164` | 0.637s - 4.153s | 55.6s - 55.2s |
| `q20_24b` | 0.501s - 5.508s | 1.5s - 1.6s |


| Query Name | Planning Times (sec) | Execution Times (sec) |
|-|-|-|
| `28b` | 0.595s - 8.222s | 2.6s - 2.6s |
| `29a` | 0.881s - 12.540s | 0.0s - 0.0s |
| `q17_7a164` | 1.114s - 4.547s | 52.6s - 52.6s |
| `q17_7a164` | 0.637s - 4.153s | 55.6s - 55.2s |
| `q20_24b` | 0.501s - 5.508s | 1.5s - 1.6s |

it means, that queries can be speed up to `10x` by simply applying a hint with faster planning

In [18]:
def print_summary_about_plans(bench_name, settings):
    oracle = cached_oracles[bench_name]
    visited_states = 0
    states_with_unique_plans = 0
    visited_unique_plans = 0 
    all_states = [SearchingState(dop, hintset) for dop in DOPS for hintset in HINTSETS]
    for query_name in oracle.get_query_names():
        explorer = QueryExplorer(query_name=query_name, oracle=oracle, settings=settings)
        dop, hintset = explorer.run()
        visited_states += len(set(explorer.explored_states))
        visited_unique_plans += len(set(explorer.explored_plans))
        states_with_unique_plans += len(set([explorer.get_plan(st) for st in all_states]))
        

    total_states = len(HINTSETS) * len(DOPS) * len(oracle.get_query_names())

    print(f"| `{bench_to_title[bench_name]}` | {visited_states} / {total_states} | {visited_unique_plans} / {states_with_unique_plans} |")

In [19]:
all_moves_settings = SearchingSettings(
        force_join=True, 
        disable_inl=True, 
        disable_ops=True, 
        use_joined_search=True, 
        decrease_dop=True,
        max_iter=float("inf")
)
for bench_name in cached_oracles:
    print_summary_about_plans(bench_name, all_moves_settings)

| `JOB` | 7980 / 43392 | 2824 / 7692 |
| `TPCH` | 1326 / 8448 | 248 / 600 |
| `SQ` | 2637 / 15360 | 1366 / 4524 |


| BENCHMARK | # STATES (VISITED / ALL) | # PLANS (VISITED / ALL) | 
|-|-|-|
| `JOB` | 7980 / 43392 | 2824 / 7692 |
| `TPCH` | 1326 / 8448 | 248 / 600 |
| `SQ` | 2637 / 15360 | 1366 / 4524 |

We see, that local search algorithm significantly reduce search space, but anyway it often executes the same plan several times. This makes no sense, because the execution time does not depend on the hints we recommend, **only the plan matters**.

## *Which features are most important to accelerate learning itself*?

Given that a) often very good solutions exist near the initial state, b) the number of unique plans is relatively small, and c) the quality of `SearchingState` is determined solely by the plan, we propose the following techniques to reduce training time:
- limit the number of iterations in local search.
- pre-plan neighbors and avoid executing the same plans repeatedly.
- use timeouts when exploring neighbors (there is no point in waiting for a request to complete if it will take longer than the best known solution).
- implement aggressive timeout
- use only subset of moves (i.e., consider only a specific part of the neighborhood, for example, limit to just turning off `INL` or decreasing `DOP`)

To determine which of these techniques are the most effective, we will introduce a *score* for the search parameters $x$ similar to the F-Score -- this metric balances between the proportion of the boost obtained from the maximum possible acceleration and the proportion of saved training time:


$$\text{score}_{\beta}(\text{x}) = F_{\beta}(\text{boost\_coeff}(\text{x}), \text{learning\_coeff}(\text{x}))$$

где $\text{boost\_coeff}(\text{x}) = \frac{\text{max\_possible\_boost}}{\text{learning\_time(x)}}$ и  $\text{learning\_coeff}(\text{x}) = \frac{\text{max\_possible\_time} - \text{learning\_time}(x)}{\text{max\_possible\_time}}$

For different $\beta$ we will find the best configurations of parameters of the local search algorithm. After that it will become clear which combinations of the proposed techniques are the most useful in each of the cases - 1) when we equally care about both boost and learning time, 2) when we want to get as much boost as possible, and 3) when our research budget is extremely limited.

In [20]:
def find_best_settings(bench_name, condition=None, beta=2):
    max_learning_time = max(learning_times[bench_name].values())
    max_speedup = def_times[bench_name] - best_e2e_times[bench_name]
    best_score, best_settings = float("-inf"), None
    
    for settings in e2e_times[bench_name]:
        if condition and not condition(settings):
            continue
        
        saved_learning_time = max_learning_time - learning_times[bench_name][settings]
        learning_coef = saved_learning_time / max_learning_time
        boost = def_times[bench_name] - e2e_times[bench_name][settings]
        assert boost >= 0
        boost_coef = boost / max_speedup
        score = (1 + beta ** 2) * learning_coef * boost_coef / (beta ** 2 * learning_coef + boost_coef)
        if score > best_score:
            best_score, best_settings = score, settings

    speedup = def_times[bench_name] - e2e_times[bench_name][best_settings]
    speedup_coef = speedup / max_speedup 
    learning_time = learning_times[bench_name][best_settings]
    
    best_settings = best_settings._asdict()
    def_settings = SearchingSettings()._asdict()
    changes = {}
    for key in best_settings:
        if best_settings[key] != def_settings[key]:
            changes[key] = best_settings[key]
            
    print(f"BOOST {speedup:0.1f}s ({100 * speedup_coef:0.1f} %) | LEARNED IN {learning_time:0.1f}s")
    print(dumps(changes, indent=4))

In [24]:
bench_name = "sample_queries"
condition = lambda el: not el.prioritize_neighbors
for beta in [1/10, 1/4, 1/2, 1/1.5, 1, 1.5, 2, 4, 10]:
    find_best_settings(bench_name, condition=condition, beta=beta)

BOOST 318.2s (55.9 %) | LEARNED IN 1164.3s
{
    "boost_threshold": 1.2,
    "max_iter": 1,
    "avoid_duplicates": true,
    "use_timeout": true,
    "force_only_nl": true
}
BOOST 403.1s (70.8 %) | LEARNED IN 1846.8s
{
    "boost_threshold": 1.5,
    "max_iter": 1,
    "avoid_duplicates": true,
    "use_timeout": true,
    "force_join": true
}
BOOST 477.9s (83.9 %) | LEARNED IN 3239.8s
{
    "decrease_dop": true,
    "disable_inl": true,
    "max_iter": 2,
    "avoid_duplicates": true,
    "use_timeout": true,
    "force_only_nl": true
}
BOOST 477.9s (83.9 %) | LEARNED IN 3239.8s
{
    "decrease_dop": true,
    "disable_inl": true,
    "max_iter": 2,
    "avoid_duplicates": true,
    "use_timeout": true,
    "force_only_nl": true
}
BOOST 477.9s (83.9 %) | LEARNED IN 3239.8s
{
    "decrease_dop": true,
    "disable_inl": true,
    "max_iter": 2,
    "avoid_duplicates": true,
    "use_timeout": true,
    "force_only_nl": true
}
BOOST 531.7s (93.3 %) | LEARNED IN 7916.8s
{
    "disable_o

**General observations:**

- with limited training resources, it is advisable to limit to a single iteration of local search and use aggressive timeouts;
- avoiding execution same plan multiple times is beneficial across all benchmarks

`JOB`:
- the greatest performance gain is achieved by managing joins while simultaneously adjusting the `DOP`
- disabling of operations turns out to be quite costly — in almost all configurations, it is more advantageous to allocate resources to exploring other aspects (`disable_ops` is `True` only with very high values of $\beta$)

`SQ`:
- disabling of operations is extremely costly too, and it is possible to manage with just reducing `DOP` and managing joins

`TPCH`:
- disabling of operations is not as costly, allowing a rapid achievement of `80%` performance boost

### *How does the order of neighbor important?*

We see that there is usually a short path from the initial solution to the suboptimal solution. If we suddenly have some algorithm that will feed us neighbors to check in an optimal order, we can reach the optimum many times faster. Let's see how fast we can approach the optimum if we have some oracle available that sorts the neighbors in the order of their actual execution. If this turns out to be a promising technique, we can try to put the responsibility for this on the neural network - there will be no consequences from incorrect predictions (no regressions, no lost gains), but the potential benefit of correct ordering is obvious.

In [25]:
bench_name = "JOB"
condition = lambda el: el.prioritize_neighbors
for beta in [1/10, 1/4, 1/2, 1/1.5, 1, 1.5, 2, 4, 10]:
    find_best_settings(bench_name, condition=condition, beta=beta)

BOOST 348.8s (97.2 %) | LEARNED IN 711.3s
{
    "disable_ops": true,
    "disable_inl": true,
    "max_iter": 1,
    "avoid_duplicates": true,
    "use_joined_search": true,
    "prioritize_neighbors": true,
    "force_join": true
}
BOOST 348.8s (97.2 %) | LEARNED IN 711.3s
{
    "disable_ops": true,
    "disable_inl": true,
    "max_iter": 1,
    "avoid_duplicates": true,
    "use_joined_search": true,
    "prioritize_neighbors": true,
    "force_join": true
}
BOOST 348.8s (97.2 %) | LEARNED IN 711.3s
{
    "disable_ops": true,
    "disable_inl": true,
    "max_iter": 1,
    "avoid_duplicates": true,
    "use_joined_search": true,
    "prioritize_neighbors": true,
    "force_join": true
}
BOOST 348.8s (97.2 %) | LEARNED IN 711.3s
{
    "disable_ops": true,
    "disable_inl": true,
    "max_iter": 1,
    "avoid_duplicates": true,
    "use_joined_search": true,
    "prioritize_neighbors": true,
    "force_join": true
}
BOOST 351.8s (98.1 %) | LEARNED IN 1011.7s
{
    "disable_ops": true

It can be seen that learning can be accelerated many times (or even tens of times). This means that the idea of ranking neighbors through a separate predictive model can be useful.